## The Challenge

As a wealth manager you have a number of investors who are associated with multiple accounts. In addition some of these investors share accounts with other investors. Together these accounts form a household for the investor/s. You would like to be able to see the holdings for an investor based on each household they are associated with. 

## The Solution

1) Create your instrument universe using a range of identifiers

2) Set up a scope to hold the accounts of a subfirm

3) Create a portfolio for each account that exists with the subfirm

4) Create a property to hold the investor and the household

5) Set the initial holdings of the portfolio

5) Create a portfolio group for each household

6) Load market data prices

7) Conduct a valuation against each household

In [18]:
# Import LUSID
import lusid.models as models
import lusid
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.5.2676.0


### Create your instrument universe using a range of identifiers

Before you can take on any holdings for your client accounts you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [19]:
equity_instruments = pd.read_csv('data/households-instruments-equities.csv')
equity_instruments.head(n=20)

,InstrumentName,ClientInternal,Currency,Figi,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,VANGUARD LIFESTRATEGY GROWTH,imd_89881022,USD,BBG000BTB8B1,US,united_states_america,VASGX,equity,fund_of_funds,NaN
3,Salesforce.com Inc,imd_44953022,USD,BBG000BN2DC2,US,united_states_america,CRM,equity,common_stock,NaN
4,Norwegian Cruise Line Holdings Ltd,imd_44569345,USD,BBG000BSRN78,US,united_states_america,NCLH,equity,common_stock,NaN
5,CAMPBELL SOUP CO,imd_44645943,USD,BBG000BG4202,US,united_states_america,CPB,equity,common_stock,NaN
6,Under Armour Inc,imd_44678322,USD,BBG009DTD8H2,US,united_states_america,UA,equity,common_stock,NaN
7,Halliburton Co,imd_44588822,USD,BBG000BKTFN2,US,united_states_america,HAL,equity,common_stock,NaN
8,MSCI Inc,imd_44999201,USD,BBG000RTDY25,US,united_states_america,MSCI,equity,common_stock,NaN
9,Brown-Forman Corp,imd_45992929,USD,BBG000BD2C18,US,united_states_america,BF/A,equity,common_stock,NaN


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [20]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in equity_instruments.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = ['Figi', 'Ticker', 'ClientInternal']
    
    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier] = models.InstrumentIdValue(
            value=instrument[identifier])
    
    # Build your request and add it to the dictionary
    batch_upsert_request[instrument['InstrumentName']] = models.InstrumentDefinition(
        name=instrument['InstrumentName'],
        identifiers=identifiers)
    
# Call LUSID to upsert your instrument defintions
instrument_response = client.instruments.upsert_instruments(
    requests=batch_upsert_request)

# Pretty print the response
prettyprint.instrument_response(instrument_response)

Instrument Successfully Upserted: Brown-Forman Corp
ClientInternal ID: imd_45992929
LUSID Instrument ID: LUID_M5ATAEQX


Instrument Successfully Upserted: VANGUARD LIFESTRATEGY GROWTH
ClientInternal ID: imd_89881022
LUSID Instrument ID: LUID_NGEM1RGY


Instrument Successfully Upserted: Under Armour Inc
ClientInternal ID: imd_44678322
LUSID Instrument ID: LUID_6IAN0BM4


Instrument Successfully Upserted: Halliburton Co
ClientInternal ID: imd_44588822
LUSID Instrument ID: LUID_9IT0U0G4


Instrument Successfully Upserted: CAMPBELL SOUP CO
ClientInternal ID: imd_44645943
LUSID Instrument ID: LUID_AV646SUT


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
ClientInternal ID: imd_34634534
LUSID Instrument ID: LUID_NASN8I1X


Instrument Successfully Upserted: MSCI Inc
ClientInternal ID: imd_44999201
LUSID Instrument ID: LUID_LKY4ZA6U


Instrument Successfully Upserted: Salesforce.com Inc
ClientInternal ID: imd_44953022
LUSID Instrument ID: LUID_3TU332OD


Instrument Successfully Upserted:

### Set up a scope to hold the accounts of a subfirm

Your wealth managment company hasa number of different branches (subfirms). You are going to start with a single subfirm as a test. You will need to create a name for the scope that you will use for the subfirm. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [21]:
scope_id = import_data.create_scope_id()
scope = 'subfirm-{}'.format(scope_id)
prettyprint.heading('Scope', scope)

Scope: subfirm-376a-720e-9576-20


### Create a portfolio for each account that exists with the subfirm

Now that you have decided on the name for your scope, you can create the portfolios to represent your client's accounts inside this scope. You will import the account details from a CSV file.

*Run the cell below to import your client's account details*

In [22]:
accounts = pd.read_csv('data/households-accounts.csv')
accounts.head(n=10)

,account_code,account_name,currency,description,primary_acount_owner_id,household_id,other_account_owner_id
0,040004-929987648,OneIntlBrokerage,USD,Primary trading account,invstr_7325jhv93,hhd_jxgru45055,NaN
1,040004-778939522,OneIntlBrokerage,USD,College fund,invstr_7325jhv93,hhd_jxgru45055,invstr_bbgj93921
2,204507-250099511,VangETF,USD,Passive exchange traded fund,invstr_a1kf37761,hhd_ab3452342,NaN
3,774001-899930233,FidelityOnlineUSEquityOptions,USD,Online US equity & option trades,invstr_bbgj93921,hhd_jxgru45055,NaN
4,400250-613060229,ActiveManagedBlckrock,USD,Actively managed high return investment account,invstr_a1kf37761,hhd_ab3452342,NaN
5,769231-448679901,MutualFund,USD,Pooled investment vehicle,invstr_7325jhv93,hhd_ab3452342,NaN


With the account details loaded you can now create your portfolios.

Note that every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolios in the cell below you are creating it with a 'created' date of 1052 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolios*

In [23]:
for index, account in accounts.iterrows():

    # Set the creation date of your portfolio 
    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

    # Build your request to create your portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=account['account_name'],
        code=account['account_code'],
        base_currency=account['currency'],
        description=account['description'],
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method='AverageCost',
        sub_holding_keys=None,
        properties=None)

    # Call LUSID to create your portfolio
    response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_request=request)

    # Pretty print the response
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: subfirm-376a-720e-9576-20
Code: 040004-929987648
Portfolio Effective From: 2016-07-19 10:16:24.603011+00:00
Portfolio Created On: 2019-06-06 10:16:24.520308+00:00

Portfolio Created
Scope: subfirm-376a-720e-9576-20
Code: 040004-778939522
Portfolio Effective From: 2016-07-19 10:16:24.919778+00:00
Portfolio Created On: 2019-06-06 10:16:24.811324+00:00

Portfolio Created
Scope: subfirm-376a-720e-9576-20
Code: 204507-250099511
Portfolio Effective From: 2016-07-19 10:16:25.203735+00:00
Portfolio Created On: 2019-06-06 10:16:25.079117+00:00

Portfolio Created
Scope: subfirm-376a-720e-9576-20
Code: 774001-899930233
Portfolio Effective From: 2016-07-19 10:16:25.456281+00:00
Portfolio Created On: 2019-06-06 10:16:25.332466+00:00

Portfolio Created
Scope: subfirm-376a-720e-9576-20
Code: 400250-613060229
Portfolio Effective From: 2016-07-19 10:16:26.755075+00:00
Portfolio Created On: 2019-06-06 10:16:27.518008+00:00

Portfolio Created
Scope: subfirm-376a-720e-9576-20
Code

### Create a property to hold the investor and the household

To keep track of the investor associated with each portfolio as well as the household you can make use of LUSID's extensible properties. These allow you to define a bespoke schema for your portfolio objects that contains this information. Read more about properties in the [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property). 

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold household_id, and account owner ids*

In [24]:
property_codes = ['household_id', 'primary_acount_owner_id', 'other_account_owner_id']

for property_code in property_codes:

    # Create our request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Portfolio',
        scope=scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope='default', code='string'))

    # Call LUSID to create our new property
    response = client.property_definitions.create_property_definition(
        definition=request)

    # Grab the key off the response to use when referencing this property in other LUSID calls
    portfolio_property_key = response.key

    # Pretty print our key
    prettyprint.heading('Portfolio Property Key', portfolio_property_key)

Portfolio Property Key: Portfolio/subfirm-376a-720e-9576-20/household_id
Portfolio Property Key: Portfolio/subfirm-376a-720e-9576-20/primary_acount_owner_id
Portfolio Property Key: Portfolio/subfirm-376a-720e-9576-20/other_account_owner_id


With these properties created you can now populate them for each portfolio.

*Run the cell below to populate the values for these properties for each portfolio*

In [25]:
# Iterate over each account
for index, account in accounts.iterrows():
    # Add the relevant account properties to the portfolio
    response = client.portfolios.upsert_portfolio_properties(
        scope=scope,
        code=account['account_code'],
        portfolio_properties={
            'Portfolio/{}/{}'.format(scope, account_property): models.PropertyValue(
                label_value=account[account_property]
            ) for account_property in property_codes
        }
    )

    # Pretty print the response 
    prettyprint.portfolio_properties_response(response)
    print ('\n')

Properties Sucessfully Updated for Portfolio
Scope:  subfirm-376a-720e-9576-20
Code:  040004-929987648 

Property key: Portfolio/subfirm-376a-720e-9576-20/household_id
Value: hhd_jxgru45055

Property key: Portfolio/subfirm-376a-720e-9576-20/other_account_owner_id
Value: NaN

Property key: Portfolio/subfirm-376a-720e-9576-20/primary_acount_owner_id
Value: invstr_7325jhv93



Properties Sucessfully Updated for Portfolio
Scope:  subfirm-376a-720e-9576-20
Code:  040004-778939522 

Property key: Portfolio/subfirm-376a-720e-9576-20/household_id
Value: hhd_jxgru45055

Property key: Portfolio/subfirm-376a-720e-9576-20/other_account_owner_id
Value: invstr_bbgj93921

Property key: Portfolio/subfirm-376a-720e-9576-20/primary_acount_owner_id
Value: invstr_7325jhv93



Properties Sucessfully Updated for Portfolio
Scope:  subfirm-376a-720e-9576-20
Code:  204507-250099511 

Property key: Portfolio/subfirm-376a-720e-9576-20/household_id
Value: hhd_ab3452342

Property key: Portfolio/subfirm-376a-720e-9

### Set the initial holdings of the portfolio

Now that you have your instrument universe populated and portfolios created you can load your client's current holdings into their portfolios. In this case you will import their holdings from a CSV file. 

*Run the cell below to import your client's current holdings*

In [26]:
holdings = pd.read_csv('data/households-holdings.csv')
holdings.head(n=20)

,portfolio_code,instrument_name,quantity,price,currency,figi
0,040004-929987648,USD_Cash,25000,1.00,USD,NaN
1,040004-929987648,Amazon_Nasdaq_AMZN,31,1538.50,USD,BBG000BVPXP1
2,040004-929987648,Salesforce.com Inc,52,152.49,USD,BBG000BN2DC2
3,040004-929987648,Norwegian Cruise Line Holdings Ltd,80,68.98,USD,BBG000BSRN78
4,040004-929987648,CAMPBELL SOUP CO,150,44.25,USD,BBG000BG4202
5,040004-929987648,Under Armour Inc,300,22.52,USD,BBG009DTD8H2
6,040004-929987648,Apple_Nasdaq_AAPL,60,182.19,USD,BBG000B9XVV8
7,040004-778939522,USD_Cash,15000,1.00,USD,NaN
8,040004-778939522,Brown-Forman Corp,250,59.86,USD,BBG000BD2C18
9,040004-778939522,Halliburton Co,39,18.54,USD,BBG000BKTFN2


Now that you have imported your client's holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to upsert your holdings into LUSID*

In [27]:
# Make the holdings effective from now
holdings_effective_date = datetime.now(pytz.UTC)

# Iterate the portfolios in the holdings CSV, note in this case you only have one
for portfolio in holdings['portfolio_code'].unique():
    # Initialise a list to hold your adjustments
    holding_adjustments = []
    
    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[holdings['portfolio_code'] == portfolio].iterrows():
        
        # Set your instrument identifiers based on whether or not instrument is cash
        if 'Cash' in holding['instrument_name']:
            identifier_key = 'Instrument/default/Currency'
            identifer = holding['instrument_name'].split('_')[0]
        else:
            identifier_key = 'Instrument/default/Figi'
            identifer = holding['figi']
            
        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=holding['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ]
            )
        )
    
    # Call LUSID to set your initial holdings
    response = client.transaction_portfolios.set_holdings(
        scope=scope,
        code=portfolio,
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(response, scope, portfolio)

Holdings Successfully Set for Portfolio
Scope: subfirm-376a-720e-9576-20
Code: 040004-929987648
Holdings Effective From: 2019-06-06 10:16:41.646714+00:00
Holdings Created On: 2019-06-06 10:16:41.880739+00:00

Holdings Successfully Set for Portfolio
Scope: subfirm-376a-720e-9576-20
Code: 040004-778939522
Holdings Effective From: 2019-06-06 10:16:41.646714+00:00
Holdings Created On: 2019-06-06 10:16:42.370643+00:00

Holdings Successfully Set for Portfolio
Scope: subfirm-376a-720e-9576-20
Code: 204507-250099511
Holdings Effective From: 2019-06-06 10:16:41.646714+00:00
Holdings Created On: 2019-06-06 10:16:42.949333+00:00

Holdings Successfully Set for Portfolio
Scope: subfirm-376a-720e-9576-20
Code: 774001-899930233
Holdings Effective From: 2019-06-06 10:16:41.646714+00:00
Holdings Created On: 2019-06-06 10:16:43.460472+00:00

Holdings Successfully Set for Portfolio
Scope: subfirm-376a-720e-9576-20
Code: 400250-613060229
Holdings Effective From: 2019-06-06 10:16:41.646714+00:00
Holdings C

### Create a portfolio group for each household

Now that you've added your client's holdings to their portfolio you want to be able to see for a given investor (in this case investor "invstr_7325jhv93") the net worth of any households that they are associated with.

To do this you can make use of portfolio groups.

Read more about portfolio groups here [LUSID Knowledge Base: How do you Group and Aggregate Portfolios?](https://support.lusid.com/how-do-you-group-and-aggregate-portfolios)

*Run the cell below to create a portfolio group for each household*

In [28]:
# Iterate over all households
for household in accounts['household_id'].unique():
    
    group_request = models.CreatePortfolioGroupRequest(
    code=household+'-Group',
    display_name='Contains all accounts for the {} household'.format(household))

    response = client.portfolio_groups.create_portfolio_group(
        scope=scope,
        request=group_request)

    prettyprint.portfolio_group_response(response, 'created')

Portfolio Group Created
Name: Contains all accounts for the hhd_jxgru45055
Scope: subfirm-376a-720e-9576-20
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the hhd_ab3452342
Scope: subfirm-376a-720e-9576-20
Code: hhd_ab3452342-Group
Portfolios Inside Group: 




You can now add the portfolios for each household to the appropriate group.

*Run the cell below to add all of the portfoltios in each household to the group*

In [48]:
portfolios = client.portfolios.list_portfolios_for_scope(
    scope=scope) 

for portfolio in portfolios.values:
    
    properties = client.portfolios.get_portfolio_properties(
        scope=portfolio.id.scope, 
        code=portfolio.id.code)
    
    portfolio_properties = {prop.key: prop.value for prop in properties.properties}
    
    response = client.portfolio_groups.add_portfolio_to_group(
        scope=scope,
        code=portfolio_properties['Portfolio/{}/household_id'.format(scope)]+'-Group',
        portfolio_id=models.ResourceId(
            scope=portfolio.id.scope,
            code=portfolio.id.code))

    prettyprint.get_portfolio_group_response(response)
    # Search for portfolios in this household
    # Add them to the portfolio group

Portfolio Group: 
Name: Contains all accounts for the hhd_jxgru45055
Scope: subfirm-376a-720e-9576-20
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_ab3452342
Scope: subfirm-376a-720e-9576-20
Code: hhd_ab3452342-Group
Portfolios Inside Group: 
400250-613060229
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_jxgru45055
Scope: subfirm-376a-720e-9576-20
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
040004-778939522
040004-929987648
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_ab3452342
Scope: subfirm-376a-720e-9576-20
Code: hhd_ab3452342-Group
Portfolios Inside Group: 
400250-613060229
769231-448679901
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the hhd_jxgru45055
Scope: subfirm-376a-720e-9576-20
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
040004-778939522
04

### Load market data prices

To value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation).

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [31]:
prices = pd.read_csv('data/households-prices.csv')
prices.head(n=50)

,instrument_name,currency,figi,price_current,ticker,client_internal
0,VANGUARD LIFESTRATEGY GROWTH,USD,BBG000BTB8B1,33.2,VASGX,imd_34634534
1,Amazon_Nasdaq_AMZN,USD,BBG000BVPXP1,"1,738.50",AMZN,imd_35345345
2,Apple_Nasdaq_AAPL,USD,BBG000B9XVV8,182.54,AAPL,imd_89881022
3,Salesforce.com Inc,USD,BBG000BN2DC2,158.44,CRM,imd_44953022
4,Norwegian Cruise Line Holdings Ltd,USD,BBG000BSRN78,52.38,NCLH,imd_44569345
5,CAMPBELL SOUP CO,USD,BBG000BG4202,41.93,CPB,imd_44645943
6,Under Armour Inc,USD,BBG009DTD8H2,25.76,UA,imd_44678322
7,Halliburton Co,USD,BBG000BKTFN2,21.19,HAL,imd_44588822
8,MSCI Inc,USD,BBG000RTDY25,234.5,MSCI,imd_44999201
9,Brown-Forman Corp,USD,BBG000BD2C18,54.1,BF/A,imd_45992929


Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to upsert the market data quotes into LUSID*

In [36]:
instrument_quotes = []

for index, quote in prices.iterrows():
    
    luid = client.search.instruments_search(
        symbols=[
            models.InstrumentSearchProperty(
                key='Instrument/default/Figi',
                value=quote['figi'])
        ],
        mastered_only=True
        )[0].mastered_instruments[0].identifiers['LusidInstrumentId'].value
    
    instrument_quotes.append(models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            provider='DataScope',
            price_source='USDRC',
            instrument_id=luid,
            instrument_id_type='LusidInstrumentId',
            quote_type='Price',
            price_side='Mid'),
        metric_value=models.MetricValue(
            value=quote['price_current'],
            unit=quote['currency']),
        effective_at=holdings_effective_date,
        lineage='InternalSystem'
    ))
    
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-06-06 10:24:22.964409+00:00


### Conduct a valuation against each household

Now that the quotes have been added to the quote store you can aggregate across each household. The logic for an aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to aggregate and value the base fund*

In [57]:
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.Figi.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
           models.MarketDataKeyRule(
               key='Equity.Isin.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
            models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
            models.MarketDataKeyRule(
               key='Fx.CurrencyPair.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Rate',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='Figi',
            default_scope=scope)
        )
    )

for household in accounts['household_id'].unique():
    
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Holding/default/Units',
            op='Sum'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value')
            
        ],
        group_by=[
            'Instrument/default/Name'
        ])

    response = client.aggregation.get_aggregation_by_group(
        scope=scope,
        code=household+'-Group',
        request=aggregation_request)

    prettyprint.heading('Household', household)
    prettyprint.aggregation_response_generic(response)
    

Household: hhd_jxgru45055
Aggregation Results: 
Sum(Holding/default/Cost): 65987.0
Sum(Holding/default/PV): 65987.0
Sum(Holding/default/Units): 65987.0
Instrument/default/Name: <Unknown>
Holding/default/SubHoldingKey: Currency=USD


Sum(Holding/default/Cost): 14965.0
Sum(Holding/default/PV): 13525.0
Sum(Holding/default/Units): 250.0
Instrument/default/Name: Brown-Forman Corp
Holding/default/SubHoldingKey: LusidInstrumentId=LUID_M5ATAEQX/USD


Sum(Holding/default/Cost): 5991.06
Sum(Holding/default/PV): 7183.41
Sum(Holding/default/Units): 339.0
Instrument/default/Name: Halliburton Co
Holding/default/SubHoldingKey: LusidInstrumentId=LUID_9IT0U0G4/USD


Sum(Holding/default/Cost): 24008.48
Sum(Holding/default/PV): 24082.88
Sum(Holding/default/Units): 152.0
Instrument/default/Name: Salesforce.com Inc
Holding/default/SubHoldingKey: LusidInstrumentId=LUID_3TU332OD/USD


Sum(Holding/default/Cost): 59596.86
Sum(Holding/default/PV): 67801.5
Sum(Holding/default/Units): 39.0
Instrument/default/Name